### 기본설정

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from consistentvideo import reference
from consistentvideo import story
from consistentvideo import video

디렉토리 설정

In [10]:
# 디렉토리 설정
# BASE_DIR = os.path.abspath(os.path.join('.'))
# UPLOAD_DIR = os.path.join(BASE_DIR, "uploads")
# os.makedirs(UPLOAD_DIR, exist_ok=True)
# file_path = os.path.join(UPLOAD_DIR, "컬투쇼버스.txt")

# 기본
ENTITY_SET_NAME = "컬투쇼버스1"
WORK_DIR = os.path.join("demo-file/", ENTITY_SET_NAME)

# 레퍼런스 생성부
SYNOPSIS_TXT_PATH = "demo-file/컬투쇼버스.txt"  # SYNOPSIS_TEXT 파일입출력(선택)
SYNOPSIS_TEXT = "시놉시스 텍스트"
REFERENCE_PATH = os.path.join(WORK_DIR, "reference")
REFERENCE_IMG_PATH = os.path.join(REFERENCE_PATH, "images")
REFERENCE_ENTITY_LIST_PATH = os.path.join(REFERENCE_PATH, "entity_list.txt")

# 씬/컷 분리부
STORY_TXT_PATH = "demo-file/컬투쇼버스.txt"  # STORY_TEXT 파일입출력(선택)
STORY_TEXT = "스토리 텍스트"
STORY_PATH = os.path.join(WORK_DIR, "story")
STORY_CUT_LIST_PATH = os.path.join(STORY_PATH, "cut.txt")

# 비디오 생성부
VIDEO_PATH = os.path.join(WORK_DIR, "video")
CUT_IMG_PATH = os.path.join(VIDEO_PATH, "cut-images")
VIDEO_OUTPUT_PATH = os.path.join(VIDEO_PATH, "output")
VIDEO_CLIP_LIST_PATH = os.path.join(VIDEO_PATH, "clip_file_list.txt")
VIDEO_CONCAT_OUTPUT_PATH = os.path.join(VIDEO_PATH, f"{ENTITY_SET_NAME}_concat_video.mp4")

os.makedirs(WORK_DIR, exist_ok=True)

## 레퍼런스 생성부 테스트

In [3]:
# 클래스 인스턴스
analyzer = reference.SynopsisAnalyzer()
character_creator = reference.CharacterImageCreator()
location_creator = reference.LocationImageCreator()
object_creator = reference.ObjectImageCreator()

In [4]:
# SYNOPSIS_TEXT 파일입출력(선택)
with open(SYNOPSIS_TXT_PATH, "r", encoding="utf-8") as f:
    SYNOPSIS_TEXT = f.read()

# 3. 시놉시스 분석 및 파싱
analyzer.save_dir = os.path.join(REFERENCE_PATH, "analyzer")
entity_dict_draft_list = analyzer.analyze(SYNOPSIS_TEXT)


In [ ]:
# 4. Creator들에게 base 디렉토리 설정
character_creator.set_base_dir(REFERENCE_IMG_PATH)
location_creator.set_base_dir(REFERENCE_IMG_PATH)
object_creator.set_base_dir(REFERENCE_IMG_PATH)

# 5. 이미지 생성 및 결과 수집
entity_list = []
# results를 txt 파일로 저장
with open(REFERENCE_ENTITY_LIST_PATH, "w", encoding="utf-8") as f:
    for entity in entity_dict_draft_list:
        type_ = entity.get("type")
        name = entity.get("name")
        description = entity.get("description")

        if name == "기타":
            result = (type_, name, description, None)
        elif type_ == "character":
            result = character_creator.create(type_, name, description)
        elif type_ == "location":
            result = location_creator.create(type_, name, description)
        elif type_ == "object":
            result = object_creator.create(type_, name, description)
        else:
            result = (type_, name, description, None)

        print(f"[RESULT] {result}")  # 터미널 출력
        entity_list.append(result)
        f.write(str(result) + "\n")


In [ ]:
print(entity_list)

## 씬/컷 분리부 테스트

In [ ]:
def load_entity_list(file_path):
    entities = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            # 빈 줄 건너뛰기
            if not line.strip():
                continue
            # 문자열을 튜플로 안전하게 변환
            try:
                entity_tuple = eval(line.strip())
                entities.append(entity_tuple)
            except Exception as e:
                print(f"라인 파싱 중 오류 발생: {e}")
                continue
    return entities


# 엔티티 리스트 로드
entity_list = load_entity_list(REFERENCE_ENTITY_LIST_PATH)
print(entity_list)

In [ ]:
# 예시 스토리
with open(STORY_TXT_PATH, "r", encoding="utf-8") as f:
    STORY_TEXT = f.read()

scene_gen = story.SceneGenerator()
scenes = scene_gen.generate_scenes(STORY_TEXT)

os.makedirs(STORY_PATH, exist_ok=True)
with open(os.path.join(STORY_PATH, "scene.txt"), "w", encoding="utf-8") as f:
    for scene in scenes:
        print(scene)
        f.write(str(scene) + "\n")


In [ ]:
cut_generator = story.CutGenerator()
cut_list = list()

with open(STORY_CUT_LIST_PATH, "w", encoding="utf-8") as f:
    for scene in scenes:
        cut = cut_generator.cut_scene(scene, entity_list)
        cut_list.append(cut)
        print(cut)
        f.write(str(cut) + "\n")


In [ ]:
# print(scenes)
print(cut_list)

## 비디오 생성부 테스트

In [3]:
os.makedirs(CUT_IMG_PATH, exist_ok=True)

In [4]:
# 파일에서 entity_list, cut_list 로드

def load_entity_list(file_path):
    entities = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            # 빈 줄 건너뛰기
            if not line.strip():
                continue
            # 문자열을 튜플로 안전하게 변환
            try:
                entity_tuple = eval(line.strip())
                entities.append(entity_tuple)
            except Exception as e:
                print(f"라인 파싱 중 오류 발생: {e}")
                continue
    return entities


from ast import literal_eval


def load_cut_list(file_path):
    cuts_by_scene = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            # 빈 줄 건너뛰기
            if not line.strip():
                continue
            try:
                # ast.literal_eval을 사용하여 더 안전하게 파싱
                scene_cuts = literal_eval(line.strip())
                cuts_by_scene.append(scene_cuts)
            except Exception as e:
                print(f"파싱 중 오류 발생: {e}")
                continue
    return cuts_by_scene

In [ ]:
# 엔티티 리스트 로드
entity_list = load_entity_list(REFERENCE_ENTITY_LIST_PATH)
print(entity_list)

# 컷 리스트 로드
cut_list = load_cut_list(STORY_CUT_LIST_PATH)
print(cut_list)

In [ ]:
scene_num = 1
for scene in cut_list:
    cut_num = 1
    for cut in scene:
        cut_image_generator = video.CutImageGenerator(scene_num=scene_num, cut=cut, output_path=CUT_IMG_PATH, entity_image_path=REFERENCE_IMG_PATH, entity=entity_list)
        filename = cut_image_generator.execute()
        print(f"save {filename}")
    scene_num += 1

In [ ]:
image_paths = []
for filename in os.listdir(CUT_IMG_PATH):
    if filename.lower().endswith('.png') or filename.lower().endswith('.jpg'):  # 대소문자 구분 없이 .png 확장자 확인
        image_paths.append(os.path.join(CUT_IMG_PATH, filename))
image_paths.sort()
image_paths

In [ ]:
video_generator = video.VideoGenerator(cut_list, VIDEO_OUTPUT_PATH, cut_image_list=image_paths)
video_generator.execute()

In [ ]:
video_clip_path_list = []
for filename in os.listdir(VIDEO_OUTPUT_PATH):
    if filename.lower().endswith('.mp4'):
        video_clip_path_list.append(os.path.join(VIDEO_OUTPUT_PATH, filename))
video_clip_path_list.sort()

print(video_clip_path_list)

In [ ]:
import ffmpeg

# results 리스트에 있는 비디오들을 합치기
final_output = VIDEO_CONCAT_OUTPUT_PATH

# 입력 비디오 스트림들을 준비
input_streams = [ffmpeg.input(video_path) for video_path in video_clip_path_list]

# 비디오 스트림들을 연결하고 출력
ffmpeg.concat(*input_streams).output(final_output).run(overwrite_output=True)